In [ ]:
import pandas as pd
from sqlalchemy import create_engine
# pd.set_option('display.max_rows', None) 
# df = pd.read_csv("/home/julien/Downloads/tgvmax.csv", sep=";")
# engine = create_engine('sqlite:///:memory:')
# df.rename(columns={"Disponibilité de places MAX JEUNE et MAX SENIOR":"DISPO"}, inplace=True)
# df.to_sql('TGVMAX', con=engine, index=False, if_exists='replace')
# df.head()

,DATE,TRAIN_NO,ENTITY,Axe,Origine IATA,Destination IATA,Origine,Destination,Heure_depart,Heure_arrivee,DISPO
0,2024-10-17,6627,PARISRHONE,SUD EST,FRPLY,FRXCC,PARIS (intramuros),LE CREUSOT MONTCEAU MONTCHANIN,17:52,19:09,NON
1,2024-10-21,4402,JOURNTELYN,IC SRO,FRBOU,FRLPE,BOURGES,LYON (intramuros),09:40,13:22,NON
2,2024-10-18,6871,JCSUDPROV,SUD EST,FRXYT,FRXSY,TOULOUSE MATABIAU,SETE,07:39,09:34,OUI
3,2024-10-21,7385,PAARRAS,NORD,FRPNO,FRQRV,PARIS (intramuros),ARRAS,18:49,19:40,OUI
4,2024-11-08,9402,MULHSTRAPA,EST,DEZPA,FREAH,OFFENBOURG/OFFENBURG,CHAMPAGNE-ARDENNE,17:41,19:28,NON


In [ ]:
query = """
SELECT aller.Destination, aller.Heure_depart, aller.Heure_arrivee, retour.Heure_depart as retour_heure_depart, retour.Heure_arrivee as retour_heure_arrivee, (24 - aller.Heure_arrivee + retour.Heure_depart) as temps_sur_place
FROM TGVMAX as aller
JOIN (SELECT *
      FROM TGVMAX 
      WHERE DATE = :date2 AND Destination = :ville AND DISPO = 'OUI' AND Axe != 'IC NUIT') as retour
ON aller.Destination = retour.Origine
WHERE aller.DATE = :date1 AND aller.DISPO = 'OUI' AND aller.Origine = :ville AND aller.Axe != 'IC NUIT'
ORDER BY (24 - aller.Heure_arrivee + retour.Heure_depart) DESC
"""

result_df = pd.read_sql(query, con=engine, params={'date1':'2024-10-19', 'date2':'2024-10-20', 'ville':'PARIS (intramuros)'})
result_df.head(1000)

,Destination,Heure_depart,Heure_arrivee,retour_heure_depart,retour_heure_arrivee,temps_sur_place
0,ARRAS,07:49,08:40,22:18,23:09,38
1,ARRAS,07:49,08:40,22:18,23:09,38
2,ARRAS,07:49,08:40,21:18,22:14,37
3,ARRAS,07:49,08:40,21:18,22:14,37
4,ARRAS,10:50,11:40,22:18,23:09,35
5,ARRAS,10:50,11:40,22:18,23:09,35
6,ARRAS,10:50,11:40,21:18,22:14,34
7,ARRAS,10:50,11:40,21:18,22:14,34
8,ARRAS,13:49,14:40,22:18,23:09,32
9,ARRAS,13:49,14:40,22:18,23:09,32


In [117]:
query = """
SELECT aller.Destination, aller.Heure_depart, aller.Heure_arrivee, retour.Heure_depart as retour_heure_depart, retour.Heure_arrivee as retour_heure_arrivee, (retour.Heure_depart - aller.Heure_arrivee) as temps_sur_place
FROM TGVMAX as aller
JOIN (SELECT *
      FROM TGVMAX 
      WHERE DATE = :date2 AND Destination = :ville AND DISPO = 'OUI' AND Axe != 'IC NUIT') as retour
ON aller.Destination = retour.Origine
WHERE aller.DATE = :date1 AND aller.DISPO = 'OUI' AND aller.Origine = :ville AND aller.Axe != 'IC NUIT' AND temps_sur_place > 0
ORDER BY temps_sur_place DESC
"""

result_df = pd.read_sql(query, con=engine, params={'date1':'2024-10-19', 'date2':'2024-10-19', 'ville':'PARIS (intramuros)'})
result_df.head(1000)

,Destination,Heure_depart,Heure_arrivee,retour_heure_depart,retour_heure_arrivee,temps_sur_place
0,ARRAS,07:49,08:40,21:18,22:14,13
1,ARRAS,07:49,08:40,21:18,22:14,13
2,ARRAS,07:49,08:40,20:18,21:10,12
3,ARRAS,07:49,08:40,20:18,21:10,12
4,BETHUNE,07:49,09:15,20:43,22:14,11
5,DOUAI,07:49,08:58,19:52,21:10,11
6,LENS,07:49,09:02,20:57,22:14,11
7,HAZEBROUCK,07:49,09:38,20:20,22:14,11
8,VICHY,07:01,09:53,20:06,22:57,11
9,MOULINS SUR ALLIER,07:01,09:25,20:32,22:57,11


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Connect to a file-based SQLite database
engine = create_engine('sqlite:///tgvmax.db')  # Will create tgvmax.db in the working directory

# Load CSV and populate the database
df = pd.read_csv("/home/julien/Downloads/tgvmax.csv", sep=";")
df.rename(columns={"Disponibilité de places MAX JEUNE et MAX SENIOR":"DISPO"}, inplace=True)
df.to_sql('TGVMAX', con=engine, index=False, if_exists='replace')

In [19]:
import requests
import datetime

# Define the base URL for the API request
url = "https://ressources.data.sncf.com/api/explore/v2.1/catalog/datasets/tgvmax/records"

# Initialize variables
all_data = []  # List to store all records
limit = 100  # You can adjust this number as needed
offset = 0
params = {
    'limit': limit,  # Number of records to retrieve per request
    'offset': offset,  # Starting point
    'timezone': 'Europe/Berlin'
}

dates = [datetime.datetime.today()+datetime.timedelta(i) for i in range(60)]
dates = list(map(lambda date: date.strftime('%Y-%m-%d'), dates))
print(dates)
for date in dates:
    while True:
        # Update the offset in params
        params['offset'] = offset

        # Make the API request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Append the retrieved records to the all_data list
            all_data.extend(data['results'])

            # Increment the offset for the next batch
            offset += limit
            # break
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            print(response.text)
            break

# Print the total number of records retrieved
print(f"Total records retrieved: {len(all_data)}")

['2024-10-18', '2024-10-19', '2024-10-20', '2024-10-21', '2024-10-22', '2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26', '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02', '2024-11-03', '2024-11-04', '2024-11-05', '2024-11-06', '2024-11-07', '2024-11-08', '2024-11-09', '2024-11-10', '2024-11-11', '2024-11-12', '2024-11-13', '2024-11-14', '2024-11-15', '2024-11-16', '2024-11-17', '2024-11-18', '2024-11-19', '2024-11-20', '2024-11-21', '2024-11-22', '2024-11-23', '2024-11-24', '2024-11-25', '2024-11-26', '2024-11-27', '2024-11-28', '2024-11-29', '2024-11-30', '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04', '2024-12-05', '2024-12-06', '2024-12-07', '2024-12-08', '2024-12-09', '2024-12-10', '2024-12-11', '2024-12-12', '2024-12-13', '2024-12-14', '2024-12-15', '2024-12-16']


In [12]:
print(data.keys())
print(data['total_count'])
print(len(data['results']))

dict_keys(['total_count', 'results'])
336909
100


In [15]:
print(len(all_data))

10000
